In [21]:
import pandas as pd
import numpy as np

import datetime

In [22]:
test = pd.read_csv('./test_data/test.csv', 
    na_values='<null>', sep='>', encoding='cp1251', 
    parse_dates=['LAST_CARD_STATUS_DT', 'LAST_CONTR_STATUS_DT'])

In [23]:
test = test[test['CONTRACT_REF'].isin(test['CONTRACT_REF'].drop_duplicates(keep=False))]

In [25]:
#test['BIRTH_DT']

In [19]:
test_soc_dem = pd.read_csv('./test_data/MICR_SMV_IQ_SD.csv', 
    na_values='<null>', sep='>', encoding='cp1251', 
    parse_dates=['BIRTH_DT'])

test_soc_dem['REP_CLID'] = test_soc_dem['REP_CLID'].dropna().str.replace(',', '')

test_soc_dem = test_soc_dem[['CONTRACT_NUMBER', 'FL_FRGN_TRIP', 'BIRTH_DT']]\
    .rename(columns={'BIRTH_DT': 'BIRTH_DATE', 'CONTRACT_NUMBER': 'CONTRACT_REF'})
    
#test = test.merge(test_soc_dem, on=['CONTRACT_REF'])    
#test['START_DATE'] = datetime.date(2018, 2, 20)

In [7]:
test.shape

(92691, 378)

In [8]:
train = pd.read_csv('./train_data/train_dataset.csv', 
    na_values='<null>', sep='>', 
    parse_dates=['START_DATE', 'LAST_CARD_STATUS_DT', 'LAST_CONTR_STATUS_DT'])

In [9]:
losted_cols = train.columns\
    .difference(test.columns)\
    .difference(['CONTROL_GROUP_FLG', 'SCENARIO_NAME_UNI', 'START_DATE', 'TARGET']).tolist()
losted_cols

['CODE_17_MAX_180',
 'CODE_17_MEAN_180',
 'CODE_17_N_180',
 'CODE_17_SUM_180',
 'CODE_26_MAX_180',
 'CODE_26_MEAN_180',
 'CODE_26_N_180',
 'CODE_26_SUM_180',
 'CODE_2_MAX_180',
 'CODE_2_MEAN_180',
 'CODE_2_N_180',
 'CODE_2_SUM_180',
 'CODE_35_MAX_180',
 'CODE_35_MEAN_180',
 'CODE_35_N_180',
 'CODE_35_SUM_180',
 'TXN_CHN_Our ATM_MAX_180',
 'TXN_CHN_Our ATM_MEAN_180',
 'TXN_CHN_Our ATM_N_180',
 'TXN_CHN_Our ATM_SUM_180',
 'TXN_TP_ATM_MAX_180',
 'TXN_TP_ATM_MEAN_180',
 'TXN_TP_ATM_N_180',
 'TXN_TP_ATM_SUM_180',
 'TXN_TP_MT_MAX_180',
 'TXN_TP_MT_MEAN_180',
 'TXN_TP_MT_N_180',
 'TXN_TP_MT_SUM_180',
 'TXN_TP_POS_MAX_180',
 'TXN_TP_POS_MAX_31',
 'TXN_TP_POS_MAX_90',
 'TXN_TP_POS_MEAN_180',
 'TXN_TP_POS_MEAN_31',
 'TXN_TP_POS_MEAN_90',
 'TXN_TP_POS_N_180',
 'TXN_TP_POS_N_31',
 'TXN_TP_POS_N_90',
 'TXN_TP_POS_SUM_180',
 'TXN_TP_POS_SUM_31',
 'TXN_TP_POS_SUM_90']

Среди нехватающих фичей только транзакционные - заполним их нулями.

In [10]:
losted_test = pd.DataFrame(
    data=np.zeros((test.shape[0], len(losted_cols))),
    index=test.index,
    columns=losted_cols
)

print(losted_test.shape)
losted_test.head()

(92691, 40)


,CODE_17_MAX_180,CODE_17_MEAN_180,CODE_17_N_180,CODE_17_SUM_180,CODE_26_MAX_180,CODE_26_MEAN_180,CODE_26_N_180,CODE_26_SUM_180,CODE_2_MAX_180,CODE_2_MEAN_180,...,TXN_TP_POS_MAX_90,TXN_TP_POS_MEAN_180,TXN_TP_POS_MEAN_31,TXN_TP_POS_MEAN_90,TXN_TP_POS_N_180,TXN_TP_POS_N_31,TXN_TP_POS_N_90,TXN_TP_POS_SUM_180,TXN_TP_POS_SUM_31,TXN_TP_POS_SUM_90
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
test = test.merge(losted_test, left_index=True, right_index=True)\
    [train.columns.difference(['CONTROL_GROUP_FLG', 'SCENARIO_NAME_UNI', 'TARGET']).tolist()]

In [12]:
print(test.shape)
test.head()

(92691, 212)


,11_AddrCity2,11_AddrCountry,12_AddrCity2,12_AddrCountry,13_AddrCity2,13_AddrCountry,BAL,BANK,BIRTH_DATE,CARD_CONTR_COUNT_0,...,TXN_TP_POS_N_180,TXN_TP_POS_N_31,TXN_TP_POS_N_90,TXN_TP_POS_SUM_180,TXN_TP_POS_SUM_31,TXN_TP_POS_SUM_90,TXN_TP_Purposeful credit_MAX_180,TXN_TP_Purposeful credit_MEAN_180,TXN_TP_Purposeful credit_N_180,TXN_TP_Purposeful credit_SUM_180
0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,BBKK,1986-12-29,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,0.0,BBKK,1968-03-11,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,МОНЧЕГОРСК,RU,МОНЧЕГОРСК,RU,МОНЧЕГОРСК,RU,0.0,BBKK,1959-12-23,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,КУДЬМА,RU,КУДЬМА,RU,КУДЬМА,RU,0.0,BBKK,1958-02-24,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,7.0,BBKK,1985-03-26,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


In [13]:
test.to_csv('./test_data/test_dataset.csv', index=False, sep='>', na_rep='<null>', encoding='utf-8')